## Importing required packages

In [1]:
# Load the required packages
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import re
from sklearn import svm,metrics
from sklearn.model_selection import train_test_split


## Inspecting Dataset

In [3]:
#Load dataset
df = pd.read_table("C:/Users/Swati/Desktop/spam filter/SMSSpamCollection.txt",header = None)

In [4]:
#Have a look at the dataset
df.head()

,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
#get some onfo about dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
0    5572 non-null object
1    5572 non-null object
dtypes: object(2)
memory usage: 43.6+ KB


In [7]:
# Store the target variable
y = df[0]

# Display the class distribution
y.value_counts()

ham     4825
spam     747
Name: 0, dtype: int64

In [8]:
# Encode the class labels as numbers
le = LabelEncoder()
y_enc = le.fit_transform(y)

In [9]:
# Store the SMS message data
raw_text = df[1]

# Text Preprocessing

###   Normalization

    - Replace email addresses with 'emailaddr'
    - Replace URLs with 'httpaddr'
    - Replace money symbols with 'moneysymb'
    - Replace phone numbers with 'phonenumbr'
    - Replace numbers with 'numbr

In [10]:
# Replace email addresses with 'emailaddr'
processed = raw_text.str.replace(r'\b[\w\-.]+?@\w+?\.\w{2,4}\b','emailaddr')

# Replace URLs with 'httpaddr'
processed = processed.str.replace(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)','httpaddr')

# Replace money symbols with 'moneysymb'
processed = processed.str.replace(r'£|\$', 'moneysymb')
    
# Replace phone numbers with 'phonenumbr'
processed = processed.str.replace(r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b','phonenumbr')
    
# Replace numbers with 'numbr'
processed = processed.str.replace(r'\d+(\.\d+)?', 'numbr')

# Remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ')

# Replace whitespace between terms with a single space
processed = processed.str.replace(r'\s+', ' ')

# Remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')

# Lowercase the corpus
processed = processed.str.lower()

###   Removing Stop words


In [11]:
# Access stop words
stop_words = nltk.corpus.stopwords.words('english')

# Remove all stop words
processed = processed.apply(lambda x: ' '.join(
    term for term in x.split() if term not in set(stop_words))
)

###  Stemming

In [ ]:
#Remove word stems using a Porter stemmer
porter = nltk.PorterStemmer()
processed = processed.apply(lambda x: ' '.join(
    porter.stem(term) for term in x.split())
)

Lets combine this into one handy function which takes into string and returns processed string.

In [13]:
def preprocess_text(messy_string):
    assert(type(messy_string) == str)
    cleaned = re.sub(r'\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', messy_string)
    cleaned = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr',cleaned)
    cleaned = re.sub(r'£|\$', 'moneysymb', cleaned)
    cleaned = re.sub(r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b','phonenumbr', cleaned)
    cleaned = re.sub(r'\d+(\.\d+)?', 'numbr', cleaned)
    cleaned = re.sub(r'[^\w\d\s]', ' ', cleaned)
    cleaned = re.sub(r'\s+', ' ', cleaned)
    cleaned = re.sub(r'^\s+|\s+?$', '', cleaned.lower())
    return ' '.join(
        porter.stem(term) 
        for term in cleaned.split()
        if term not in set(stop_words)
    )

Lets check this function on one example :


In [15]:
example = """  ***** CONGRATlations **** You won 2 tIckETs to Hamilton in 
NYC http://www.hamiltonbroadway.com/J?NaIOl/event   wORtH over $500.00...CALL 
555-477-8914 or send message to: hamilton@freetix.com to get ticket !! !  """

preprocess_text(example)

u'congratl numbr ticket hamilton nyc httpaddr worth moneysymbnumbr call phonenumbr send messag emailaddr get ticket'

## Feature Engineering

### Tokenization
### Implementing the tf-idf statistic
 

Finally, we're equipped to transform a corpus of text data into a matrix of numbers with one row per training example and one column per n-gram

In [17]:
# Construct a design matrix using an n-gram model and a tf-idf statistics
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_ngrams = vectorizer.fit_transform(processed)

#Let's take a look at the dimensions of the X_ngrams matrix.
X_ngrams.shape

(5572, 36345)

## Training and evaluating a model

We'll be using SVM (Support Vector Machine) which basically attempts to find hyperplane that best seperates the two classes. We'll use SVM with linear kernal because of large number of features in our training dataset. 

In [18]:
# Prepare the training and test sets using an 80/20 split
X_train, X_test, y_train, y_test = train_test_split(
    X_ngrams,
    y_enc,
    test_size=0.2,
    random_state=42,
    stratify=y_enc
)

# Train SVM with a linear kernel on the training set
clf = svm.LinearSVC(loss='hinge')
clf.fit(X_train, y_train)

# Evaluate the classifier on the test set
y_pred = clf.predict(X_test)

# Compute the F1 score
metrics.f1_score(y_test, y_pred)

0.9285714285714286

In [20]:
# Display a confusion matrix
pd.DataFrame(
    metrics.confusion_matrix(y_test, y_pred),
    index=[['actual', 'actual'], ['spam', 'ham']],
    columns=[['predicted', 'predicted'], ['spam', 'ham']]
)

predicted     
                 spam  ham
actual spam       965    1
       ham         19  130

Lets create a function that takes message as input and tells us whether the message is spam or ham.

In [23]:
def spam_filter(message):
    if clf.predict(vectorizer.transform([preprocess_text(message)])):
        return 'spam'
    else:
        return 'not spam'

In [24]:
# check this example on the above function.
spam_filter(example)

'spam'

In [26]:
# Lets check another one :
spam_filter('You have an interview tomorrow')

'not spam'